# Ring strain workflow (All sizes): PySCF DFT + PCM
This notebook loads structures saved by `ring_strain_generate_all_sizes.ipynb` and computes
T=0 K electronic energies using **PySCF DFT + PCM** (no dispersion).

We compute
\[E_{\mathrm{strain}}(b) = (E(M_{\mathrm{open}})-E(M)) - (E(X_{\mathrm{open}})-E(X)).\]

Solvent: DMF by default (ε = 36.7), changeable below.

Implementation note: this notebook uses your preferred pattern `mf = dft.RKS(mol); mf = mf.PCM();`
and falls back to `from pyscf.solvent import pcm; mf = pcm.PCM(mf)` if `.PCM()` is unavailable.


In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
import rdkit
print('RDKit', rdkit.__version__)


RDKit 2025.09.3


In [2]:
from pyscf import gto, dft
print('PySCF imported')


PySCF imported


## 1) Load manifest and choose what to run

In [3]:
out_root = Path('ring_strain_outputs_all_sizes')
manifest_path = out_root / 'manifest.json'
manifest = json.loads(manifest_path.read_text())
print('Loaded entries:', len(manifest))

# Filter: run only a specific ring (set to None to run all entries)
RING_FILTER = None  # e.g. 'medium_cyclododecane'
to_run = [e for e in manifest if (RING_FILTER is None or e['ring'] == RING_FILTER)]
print('Entries to run:', len(to_run))


Loaded entries: 3
Entries to run: 3


## 2) DFT + PCM settings

In [4]:
# DFT settings
xc = 'b3lyp'          # user-preferred default; try 'pbe0' as a no-dispersion alternative
basis = 'def2-svp'    # as requested
charge = 0
spin = 0              # 2S

# Solvent settings
pcm_method = 'IEF-PCM'   # options depend on your PySCF build; IEF-PCM is a safe default
eps = 36.7               # DMF dielectric; set 78.3553 for water

# SCF controls
max_cycle = 200
conv_tol = 1e-9


## 3) Read XYZ (first frame) -> PySCF atom string

In [5]:
def read_xyz_first_frame(xyz_path: Path):
    lines = xyz_path.read_text().splitlines()
    n = int(lines[0].strip())
    comment = lines[1].strip()
    atoms = []
    for ln in lines[2:2+n]:
        s, x, y, z = ln.split()[:4]
        atoms.append((s, float(x), float(y), float(z)))
    return comment, atoms

def atoms_to_pyscf_atom(atoms):
    return '\n'.join([f"{s} {x} {y} {z}" for (s,x,y,z) in atoms])


## 4) Single-point DFT + PCM energy (your PCM pattern + fallback)

In [6]:
def attach_pcm(mf, method: str, eps: float):
    """Attach PCM using mf.PCM() when available; otherwise fall back to pcm.PCM(mf)."""
    if hasattr(mf, 'PCM'):
        mf2 = mf.PCM()
    else:
        from pyscf.solvent import pcm
        mf2 = pcm.PCM(mf)
    mf2.with_solvent.method = method
    mf2.with_solvent.eps = float(eps)
    return mf2

def dft_pcm_energy(xyz_path: Path,
                   xc: str, basis: str, charge: int, spin: int,
                   pcm_method: str, eps: float,
                   max_cycle: int = 200, conv_tol: float = 1e-9):
    comment, atoms = read_xyz_first_frame(xyz_path)
    mol = gto.M(
        atom=atoms_to_pyscf_atom(atoms),
        basis=basis,
        charge=charge,
        spin=spin,
        unit='Angstrom',
        verbose=0,
    )
    mf = dft.RKS(mol)
    mf.xc = xc
    mf.max_cycle = max_cycle
    mf.conv_tol = conv_tol
    mf = attach_pcm(mf, method=pcm_method, eps=eps)
    e = mf.kernel()
    return float(e), comment


## 5) Run energies and compute strain

In [7]:
HARTREE_TO_KCALMOL = 627.509474

rows = []
for e in to_run:
    ring = e['ring']
    bond_tag = e['bond_tag']
    bond_idx = e['bond_idx']

    energies = {}
    for sp in ['M','M_open','X','X_open']:
        xyz_path = Path(e['species'][sp]['xyz'])
        # handle relative paths
        if not xyz_path.exists():
            xyz_path = out_root / xyz_path
        E, _ = dft_pcm_energy(
            xyz_path,
            xc=xc, basis=basis, charge=charge, spin=spin,
            pcm_method=pcm_method, eps=eps,
            max_cycle=max_cycle, conv_tol=conv_tol,
        )
        energies[sp] = E
        print(f"{ring} {bond_tag} {sp}: E = {E:.10f} Ha")

    E_strain = (energies['M_open'] - energies['M']) - (energies['X_open'] - energies['X'])

    rows.append({
        'ring': ring,
        'bond_tag': bond_tag,
        'bond_idx': bond_idx,
        'xc': xc,
        'basis': basis,
        'pcm_method': pcm_method,
        'eps': eps,
        'E_M': energies['M'],
        'E_M_open': energies['M_open'],
        'E_X': energies['X'],
        'E_X_open': energies['X_open'],
        'E_strain_Ha': E_strain,
        'E_strain_kcalmol': E_strain * HARTREE_TO_KCALMOL,
    })

df = pd.DataFrame(rows).sort_values(['ring','bond_idx']).reset_index(drop=True)
df


KeyboardInterrupt: 

## 6) Save results

In [ ]:
results_path = out_root / 'dft_pcm_results.csv'
df.to_csv(results_path, index=False)
print('Saved:', results_path.resolve())


## Optional: geometry optimization
If you want to optimize geometries in PySCF, you can use `pyscf.geomopt.geometric_solver`.
This is optional and can be expensive for larger systems.


In [ ]:
# from pyscf.geomopt.geometric_solver import optimize
#
# def optimize_dft_pcm(xyz_path: Path, xc: str, basis: str, charge: int, spin: int,
#                      pcm_method: str, eps: float):
#     comment, atoms = read_xyz_first_frame(xyz_path)
#     mol = gto.M(atom=atoms_to_pyscf_atom(atoms), basis=basis, charge=charge, spin=spin, unit='Angstrom', verbose=4)
#     mf = dft.RKS(mol)
#     mf.xc = xc
#     mf = attach_pcm(mf, method=pcm_method, eps=eps)
#     mol_opt = optimize(mf)
#     return mol_opt
#
# # Example:
# # mol_opt = optimize_dft_pcm(Path(to_run[0]['species']['M']['xyz']), xc, basis, charge, spin, pcm_method, eps)
